# Evaluation

The following notebook presents a rough outline of the required pipeline for robustness evaluation. <br />
A runnable codebase with working experiments will be soon released!

In [ ]:
from medmnistc.dataset import CorruptedMedMNIST
from medmnistc.eval import Evaluator
from medmnistc.corruptions.registry import CORRUPTIONS_DS

from torch.utils.data import DataLoader

In [ ]:
def inference(config, test_loader):
    """Placeholder for the inference pipeline"""
    # load ckpt based on config
    # ...

    # inference
    # ... 

    # return probabilities
    pass


config = {
    '...' : '...' # define here all config results
}

# Load clean dataset
test_dataset_clean = ... 
test_loader_clean = ...

# Init the Evaluator class
corruptions = CORRUPTIONS_DS[config['dataset']]
evaluator = Evaluator(dataset_name=config['dataset'],
                      true_labels=test_dataset_clean.labels,
                      corruption_types=corruptions.keys(),
                      output_folder=config['logs_path'],
                      architecture=config['architecture'],
                      task=config['task'],
                      suffix_log=f"s{config['seed']}")


# Evaluate performance on clean test set (to work with the relative corruptions)
print("\tEvaluating performance on the clean test set...")

y_pred = inference(config, test_loader_clean)


evaluator.evaluate_clean(y_pred.cpu().numpy())

# Iterate over the designed corruptions.
for corruption in corruptions.keys():

    print(corruption)
    
    # Load the corrupted test set, according to the selected corruption
    corrupted_test_test = CorruptedMedMNIST(
                                dataset_name = config['dataset'], 
                                corruption = corruption,
                                root = config['dataset_corrupted_path'],
                                mmap_mode='r')
    
    # Get dataloader
    test_loader = DataLoader(corrupted_test_test, batch_size=config['batch_size_eval'], shuffle=False)

    # Evaluate
    y_pred = inference(config, test_loader)     

    # Calculate the error
    evaluator.evaluate(y_pred.cpu().numpy(), corruption)


# Create a json file containing the aggregated and raw results
evaluator.dump_summary()